In [1]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [3]:
y=df['EColi.']
X=df.loc[:, ['陰影起伏', '曲率', '傾斜角', '傾斜方向','高度', '近くの川への距離', '鉄道ポイントへの距離', '主要道路への距離', 
             'near_distance_syorizyou', 'supply_hours', 'no water days', 'total population ', 'population served', 'popu-served', 
             'number taps', 'pipes(km)', 'pipes/tabs \n個/km', 'served/pipes', '(popu-served)/pipes\n  /km', 
             'Age of oldeset\npipe ', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [5]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [6]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [7]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

# SVC

In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    svm=SVC(kernel='linear', C=param_range[i], random_state=1)
    svm.fit(X_train_std, y_train)
    score1 = svm.score(X_test_std, y_test)
    score2 = svm.score(X_train_std, y_train)
    dic[svm]=score1

    if score1>max_score:
        val=cross_val_score(estimator=svm, X=X_train_std, y=y_train, cv=3)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(svm, score2, score1, val1))

params: SVC(C=0.0001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: SVC(C=0.001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: SVC(C=0.01, kernel='linear', random_state=1), train: 0.605263, test: 0.545455, val: 0.572549
params: SVC(C=0.1, kernel='linear', random_state=1), train: 0.638158, test: 0.590909, val: 0.611765
params: SVC(kernel='linear', random_state=1), train: 0.723684, test: 0.636364, val: 0.572026
params: SVC(C=10.0, kernel='linear', random_state=1), train: 0.697368, test: 0.651515, val: 0.546144
params: SVC(C=100.0, kernel='linear', random_state=1), train: 0.717105, test: 0.666667, val: 0.546275
params: SVC(C=1000.0, kernel='linear', random_state=1), train: 0.703947, test: 0.636364, val: 0.559477


# ロジスティック回帰

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    lr=LogisticRegression(C=param_range[i], random_state=1)
    lr.fit(X_train_std, y_train)
    score1 = lr.score(X_test_std, y_test)
    score2 = lr.score(X_train_std, y_train)
    dic[lr]=score1

    if score1>max_score:
        val=cross_val_score(estimator=lr, X=X_train_std, y=y_train, cv=3)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(lr, score2, score1, val1))

params: LogisticRegression(C=0.0001, random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: LogisticRegression(C=0.001, random_state=1), train: 0.552632, test: 0.530303, val: 0.552680
params: LogisticRegression(C=0.01, random_state=1), train: 0.605263, test: 0.530303, val: 0.578954
params: LogisticRegression(C=0.1, random_state=1), train: 0.657895, test: 0.560606, val: 0.565359
params: LogisticRegression(random_state=1), train: 0.677632, test: 0.621212, val: 0.572157
params: LogisticRegression(C=10.0, random_state=1), train: 0.717105, test: 0.651515, val: 0.572288


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

params: LogisticRegression(C=100.0, random_state=1), train: 0.717105, test: 0.606061, val: 0.578954
params: LogisticRegression(C=1000.0, random_state=1), train: 0.723684, test: 0.590909, val: 0.572810


# random forest

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.77
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1

               if score1>max_score:
                    val=cross_val_score(estimator=forest,
                                        X=X_train_std, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, score: %3f, val: %3f' %(forest, score1, val1))



params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=17,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=21,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=25,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=29,
                       random_state=1), score: 0.772727, val: 0.598431
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=36,
                       random_state=1), score: 0.772727, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1), score: 0.772727, val: 0.598954
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimato

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.77
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    score2 = forest.score(X_train_std, y_train)
                    dic[forest]=score1

               if score1>max_score:
                    val=cross_val_score(estimator=forest,
                                        X=X_train_std, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, train: %3f, test: %3f, val: %3f' %(forest, score2, score1, val1))

params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=17,
                       random_state=1), train: 0.993421, test: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=21,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=29,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.598431
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.605229
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1), train: 1.000000, test: 0.77

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.77
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1

               if score1>max_score:
                    val=cross_val_score(estimator=forest,
                                        X=X_train_std, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, score: %3f, val: %3f' %(forest, score1, val1))



params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=17,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=21,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=25,
                       random_state=1), score: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=29,
                       random_state=1), score: 0.772727, val: 0.598431
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=36,
                       random_state=1), score: 0.772727, val: 0.605229
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1), score: 0.772727, val: 0.605490
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimato

KeyboardInterrupt: 

# Boosted forest

In [31]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    score2 = ada.score(X_train_std, y_train)
                    dic[ada]=score1

                    print('params: %s, train: %3f, test: %3f, val: %3f' %(ada, score2, score1, val1))



params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=1, random_state=1), train: 0.796053, test: 0.636364, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=2, random_state=1), train: 0.828947, test: 0.621212, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=3, random_state=1), train: 0.907895, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=4, random_state=1), train: 0.980263, test: 0.696970, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=5, random_state=1), train: 1.000000, test: 0.606061, val: 0.566144
params: AdaBoos

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=44, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=45, random_state=1), train: 1.000000, test: 0.696970, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=46, random_state=1), train: 1.000000, test: 0.681818, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=47, random_state=1), train: 1.000000, test: 0.681818, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=48, random_state=1), train: 1.000000, test: 0.681818, val: 0.566144
params: Ad

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=86, random_state=1), train: 1.000000, test: 0.621212, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=87, random_state=1), train: 1.000000, test: 0.621212, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=88, random_state=1), train: 1.000000, test: 0.636364, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=89, random_state=1), train: 1.000000, test: 0.636364, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=90, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: Ad

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=33, random_state=1), train: 1.000000, test: 0.621212, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=34, random_state=1), train: 1.000000, test: 0.621212, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=35, random_state=1), train: 1.000000, test: 0.636364, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=36, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=37, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estim

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=79, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=80, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=81, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=82, random_state=1), train: 1.000000, test: 0.651515, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=83, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estim

...

                   n_estimators=5, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=6, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=7, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=8, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=9, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=10, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: Ad

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=53, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=54, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=55, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=56, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=57, random_state=1), train: 1.000000, test: 0.590909, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estim

                   learning_rate=1.5, n_estimators=2, random_state=1), train: 1.000000, test: 0.515152, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=3, random_state=1), train: 1.000000, test: 0.560606, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=4, random_state=1), train: 1.000000, test: 0.575758, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=5, random_state=1), train: 1.000000, test: 0.621212, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=6, random_state=1), train: 1.000000, test: 0.545455, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
               

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=44, random_state=1), train: 1.000000, test: 0.681818, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=45, random_state=1), train: 1.000000, test: 0.712121, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=46, random_state=1), train: 1.000000, test: 0.681818, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=47, random_state=1), train: 1.000000, test: 0.696970, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=48, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: Ad

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=86, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=87, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=88, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=89, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=90, random_state=1), train: 1.000000, test: 0.666667, val: 0.566144
params: Ad

In [17]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.65
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    dic[ada]=score1

               if score1>max_score:
                    val=cross_val_score(estimator=ada,
                                        X=X_train_std, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, score: %3f, val: %3f' %(ada, score1, val1))


params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=99, random_state=1), score: 0.651515, val: 0.565882
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=1.5, n_estimators=99, random_state=1), score: 0.666667, val: 0.552418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   n_estimators=99, random_state=1), score: 0.727273, val: 0.558824
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=99, random_state=1), score: 0.666667, val: 0.546797
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),
                   learning_rate=1.5, n_estimators=99, random_state=1), score: 0.651515, val: 0.467190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimator

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    dic[ada]=score1
                    
                    val=cross_val_score(estimator=ada,
                                        X=X_train_std, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, score: %3f, val: %3f' %(ada, score1, val1))

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=1, random_state=1), score: 0.636364, val: 0.533072
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=2, random_state=1), score: 0.621212, val: 0.526536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=3, random_state=1), score: 0.666667, val: 0.526536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=4, random_state=1), score: 0.696970, val: 0.539346
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=5, random_state=1), score: 0.606061, val: 0.499739
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=47, random_state=1), score: 0.681818, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=48, random_state=1), score: 0.681818, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=49, random_state=1), score: 0.681818, val: 0.506405
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, random_state=1), score: 0.681818, val: 0.506275
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=51, random_state=1), score: 0.681818, val: 0.506275
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=93, random_state=1), score: 0.651515, val: 0.506144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=94, random_state=1), score: 0.651515, val: 0.506144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=95, random_state=1), score: 0.651515, val: 0.506144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=96, random_state=1), score: 0.651515, val: 0.499477
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=97, random_state=1), score: 0.636364, val: 0.512680
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
           

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=44, random_state=1), score: 0.666667, val: 0.592026
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=45, random_state=1), score: 0.666667, val: 0.592026
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=46, random_state=1), score: 0.651515, val: 0.572418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=47, random_state=1), score: 0.651515, val: 0.572418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=48, random_state=1), score: 0.666667, val: 0.565882
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=49, random_state=1), score: 0.681818, val: 0.565882
params: AdaBoostClassi

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=95, random_state=1), score: 0.666667, val: 0.565752
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=96, random_state=1), score: 0.651515, val: 0.572418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=97, random_state=1), score: 0.666667, val: 0.559216
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=98, random_state=1), score: 0.651515, val: 0.585621
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=99, random_state=1), score: 0.651515, val: 0.565882
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=1.5, n_estimators=1, random_state=1), score: 0.636364, val: 0.533072
para

...

                   learning_rate=0.5, n_estimators=54, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=55, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=56, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=57, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=58, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=59, random_state=1), score: 0.530303, v

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=3, random_state=1), score: 0.621212, val: 0.519739
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=4, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=5, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=6, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=7, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=8, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(b

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=56, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=57, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=58, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=59, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=60, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=61, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassi

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=9, random_state=1), score: 0.621212, val: 0.440654
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=10, random_state=1), score: 0.651515, val: 0.447320
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=11, random_state=1), score: 0.621212, val: 0.453856
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=12, random_state=1), score: 0.666667, val: 0.447190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=13, random_state=1), score: 0.666667, val: 0.467190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
            

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=55, random_state=1), score: 0.651515, val: 0.487190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=56, random_state=1), score: 0.651515, val: 0.500523
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=57, random_state=1), score: 0.651515, val: 0.507190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=58, random_state=1), score: 0.621212, val: 0.507190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=59, random_state=1), score: 0.651515, val: 0.507190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
           